 Introdução à Base de Dados do Airbnb

O conjunto de dados "Inside Airbnb", disponível no website "http://insideairbnb.com/", é uma valiosa fonte de informações sobre listagens de hospedagem, avaliações de hóspedes e disponibilidade de calendário em várias cidades ao redor do mundo, incluindo o Rio de Janeiro. Antes de prosseguirmos com a engenharia de dados, é importante entender os principais componentes deste conjunto de dados:

1. **Listing (Listagem):** Este conjunto de dados contém informações detalhadas sobre as propriedades listadas no Airbnb. Cada registro representa uma listagem individual e inclui informações como o tipo de propriedade, preço, localização, número de quartos, comodidades oferecidas e muito mais.

2. **Reviews (Avaliações):** O conjunto de dados de avaliações contém informações sobre as avaliações feitas por hóspedes que ficaram nas propriedades listadas. Ele inclui dados como a data da avaliação, o identificador da propriedade, os comentários escritos pelos hóspedes, e outras informações. 

3. **Calendar (Calendário):** Este conjunto de dados contém informações sobre a disponibilidade das propriedades ao longo do tempo. Ele lista as datas em que as propriedades estão disponíveis para reserva, bem como os preços para cada data.

O dicionário dos dados também está disponível no website: "http://insideairbnb.com/".

 Passos do Projeto

1. **Aquisição de Dados e Armazenamento de Dados em PostgreSQL - Camada Bronze**
   - Baixe o conjunto de dados "Inside Airbnb" do Rio de Janeiro da fonte oficial (http://insideairbnb.com/) e promova uma estruturação simples nos dados.
   - Crie um banco de dados PostgreSQL para armazenar os dados brutos das 3 tabelas ("Listing", "Reviews" e Calendar") na camada "bronze".

<br>

2. **Data Clean - Camada Silver:**
   - Identifique e lide com valores ausentes, duplicatas e outliers nos dados brutos da camada "bronze".
   - Padronize e limpe os nomes das colunas, convertendo-os em um formato consistente.
   - Realize uma limpeza textual em campos, como descrições de propriedades, removendo caracteres especiais e erros de digitação.

<br>

3. **Data Quality - Camada Silver:**
   - Defina métricas de qualidade de dados, como integridade, precisão e consistência para os dados da camada "bronze".
   - Implemente verificações para garantir que os dados da camada "silver" estejam em conformidade com essas métricas.
   - Estabeleça um sistema de monitoramento contínuo da qualidade dos dados da camada "silver".

<br>

4. **Testes de Qualidade - Camada Silver:**
   - Utilize a biblioteca Great Expectations para criar testes de qualidade automatizados que verifiquem as expectativas definidas para os dados da camada "silver".
   - Desenvolva testes que assegurem que os dados da camada "silver" atendam às regras de negócios e aos requisitos de qualidade.

<br>

5. **Transformação de Dados com dbt - Camada Silver:**
   - Utilize a ferramenta dbt para criar a camada "silver" de dados, realizando transformações e preparando os dados da camada em questão.
   - Mantenha um controle de versão dos modelos dbt relacionados à camada "silver" e automatize a execução das transformações.

<br>

6. **Armazenamento de Dados em PostgreSQL - Camada Silver:**
   - Armazene os dados da camada "silver" no mesmo banco de dados PostgreSQL.
   - Estabeleça conexões entre o dbt e o PostgreSQL para carregar os dados transformados da camada "silver" no banco.

<br>

7. **Validação de Expectativas com Great Expectations - Camada Silver:**
   - Implemente validações adicionais usando Great Expectations nas camadas de dados da camada "silver".
   - Monitore a qualidade dos dados da camada "silver" após cada transformação e ajuste os testes de acordo.

<br>

8. **Transformação de Dados com dbt - Camada Gold:**
   - Utilize o dbt para criar a camada "gold" de dados, aplicando agregações especializadas, como médias de preços por propriedade, por período, e outras agregações especializadas.
   - Mantenha um controle de versão dos modelos dbt relacionados à camada "gold" e automatize a execução das transformações.
   - Armazene os dados da camada "gold" no mesmo banco de dados PostgreSQL, mantendo a estrutura de dados otimizada para consultas analíticas.

<br>

 9. **Apresentação e Discussão:**
    - Apresente os resultados do projeto para a turma, enfatizando os aspectos de engenharia de dados, qualidade de dados e uso de ferramentas como dbt, Great Expectations e o armazenamento em um banco de dados PostgreSQL nas camadas "bronze", "silver" e "gold".


### 1. **Aquisição de Dados e Armazenamento de Dados em PostgreSQL - Camada Bronze**
    - Baixe o conjunto de dados "Inside Airbnb" do Rio de Janeiro da fonte oficial (http://insideairbnb.com/) e promova uma estruturação simples nos dados.
    - Crie um banco de dados PostgreSQL para armazenar os dados brutos das 3 tabelas ("Listing", "Reviews" e Calendar") na camada "bronze".

In [149]:
!pip freeze > requirements.txt

In [51]:
import os
import requests
import pandas as pd
import numpy as np
import psycopg2 as pg
import spacy
import urllib.parse
import re
import great_expectations as gx


from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from spellchecker import SpellChecker
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from langdetect import detect

# Retire o limiter de quantidade de colunas do display do dataframe
pd.set_option('display.max_columns', None)

# Carrega as variáveis do arquivo .env
load_dotenv()

HOST = os.getenv('HOST')
PORT = os.getenv('PORT')
USER = os.getenv('USER')
PASSWORD = os.getenv('PASSWORD')

In [8]:
# URL da página
url = "http://insideairbnb.com/get-the-data"

# Faz uma requisição à página
response = requests.get(url)

# Verifica se a requisição foi bem-sucedida
if response.status_code == 200:
    # Cria um objeto BeautifulSoup para analisar o HTML
    soup = BeautifulSoup(response.content, "html.parser")

    # Encontra todos os links na página
    links = soup.find_all("a")

    # Itera pelos links
    for link in links:
        # Obtém o o nome do arquivo
        texto_link = link.text.strip()

        # Verifica se o link contém "listings.csv.gz" e "rio-de-janeiro"
        if ("listings.csv.gz" in link or "calendar.csv.gz" in link or "reviews.csv.gz" in link) and "rio-de-janeiro" in str(link):
            # Obtém o URL completo
            link_completo = urllib.parse.urljoin(url, link['href'])

            # Faz o download do arquivo
            response = requests.get(link_completo)
            
            # Cria o file_path
            file_path = 'dados/' + texto_link            

            # Verifica se o download foi bem-sucedido
            if response.status_code == 200:
                # Salva o arquivo no disco
                with open(file_path, 'wb') as f:
                    f.write(response.content)
                print(f"Arquivo '{texto_link}' baixado com sucesso.")
            else:
                print(f"Falha ao baixar o arquivo '{texto_link}'. Status code: {response.status_code}")
else:
    print(f"Falha ao acessar a página. Status code: {response.status_code}")


Arquivo 'listings.csv.gz' baixado com sucesso.
Arquivo 'calendar.csv.gz' baixado com sucesso.
Arquivo 'reviews.csv.gz' baixado com sucesso.


In [30]:
# Carrega o arquivo como DataFrame
df_listings = pd.read_csv(r"dados/listings.csv.gz", compression='gzip', low_memory=False)
df_reviews = pd.read_csv(r"dados/reviews.csv.gz", compression='gzip', low_memory=False)
df_calendar = pd.read_csv(r"dados/calendar.csv.gz", compression='gzip', low_memory=False)

display(df_listings.head(2))
display(df_reviews.head(2))
display(df_calendar.head(2))

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,231497,https://www.airbnb.com/rooms/231497,20230922043705,2023-09-22,city scrape,Rental unit in Rio de Janeiro · ★4.73 · 1 bedr...,"This is a big studio at the end of Copacabana,...",NaN,https://a0.muscache.com/pictures/3582382/ee8ac...,1207700,...,4.92,4.90,4.65,NaN,f,4,4,0,0,0.54
1,231516,https://www.airbnb.com/rooms/231516,20230922043705,2023-09-22,city scrape,Rental unit in Rio de Janeiro · ★4.71 · 1 bedr...,"Special location of the building, on Copacaban...",NaN,https://a0.muscache.com/pictures/3671683/d74b4...,1207700,...,4.86,4.93,4.38,NaN,f,4,4,0,0,0.20


,listing_id,id,date,reviewer_id,reviewer_name,comments
0,231497,726625,2011-11-17,1356783,Jenny,Es un apartastudio muy bien ubicado. Tiene lo ...
1,17878,64852,2010-07-15,135370,Tia,This apartment is in a perfect location -- two...


,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,17878,2023-09-23,f,$265.00,$265.00,5.0,28.0
1,17878,2023-09-24,t,$265.00,$265.00,5.0,28.0


In [ ]:
connection_string = f'postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}'

# Cria a engine
conection = create_engine(url)
print(f"Conexão: {conection}")

# Carrega os dados no banco
###try:
###    status = df_listings.to_sql(name='grupo_fantastico_listings', schema='raw', con=conection, if_exists='replace', index=False)
###    print(f'Status da operação: {status}')
###except Exception as e:
###    print(f'Ocorreu um erro: {e}')
###    
###try:
###    status = df_reviews.to_sql(name='grupo_fantastico_reviews', schema='raw', con=conection, if_exists='replace', index=False)
###    print(f'Status da operação: {status}')
###except Exception as e:
###    print(f'Ocorreu um erro: {e}')
###    
###try:
###    status = df_calendar.to_sql(name='grupo_fantastico_calendar', schema='raw', con=conection, if_exists='replace')
###    print(f'Status da operação: {status}')
###except Exception as e:
###    print(f'Ocorreu um erro: {e}')
###

### 2. **Data Clean - Camada Silver:**
   - Identifique e lide com valores ausentes, duplicatas e outliers nos dados brutos da camada "bronze".
   - Padronize e limpe os nomes das colunas, convertendo-os em um formato consistente.
   - Realize uma limpeza textual em campos, como descrições de propriedades, removendo caracteres especiais e erros de digitação.

 Funções para tratamento de texto

In [123]:
#!python -m spacy download pt_core_news_sm
#!python -m spacy download en_core_web_sm

In [150]:
"""
Formata os nomes das colunas de um DataFrame.
- Converte para minúsculas
- Substitui espaços por underscores
- Remove caracteres especiais

Args:
    df (pd.DataFrame): DataFrame com os nomes das colunas a serem formatados.
    
Returns:
    pd.DataFrame: DataFrame com os nomes de coluna formatados.
"""
def format_column_name(df):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(' ', '_')
    df.columns = df.columns.str.replace('[^a-zA-Z0-9_]', '', regex=True)
    return df


# Carrega o modelo de língua portuguesa
nlp = spacy.load('pt_core_news_sm')

# Cria a instância do corretor ortográfico
spell = SpellChecker(language='pt')

"""
Limpa o texto removendo caracteres especiais e corrigindo erros de digitação.

Args:
    texto (str): O texto a ser limpo.
    
Returns:
    str: Texto limpo.
"""
def clean_text(texto):
    try:
        # Processar o texto com o spaCy
        doc = nlp(texto)
        
        # Extrair tokens e verificar se estão corretos
        tokens_corrigidos = [spell.correction(token.text) for token in doc]
        
        # Juntar os tokens corrigidos de volta em uma string
        texto_corrigido = ' '.join(tokens_corrigidos)
        
        # Remove caracteres especiais
        # bug : essa linha também remove as pontuações
        texto_limpo = re.sub(r'[^a-zA-Z0-9\s]', '', texto_corrigido)
        
        return texto_limpo
    except TypeError:
        return np.nan
    

def detect_languages(df, column_name):
    # Obtém uma lista única de comentários da coluna especificada
    comments = df[column_name].dropna().unique()

    # Inicializa um conjunto para armazenar os idiomas detectados
    detected_languages = set()

    for comment in comments:
        try:
            language = detect(comment)
            detected_languages.add(language)
        except:
            pass
    
    return detected_languages


##### df_listings

Formatação do nome das colunas

In [9]:
df_listings = format_column_name(df_listings)

Conversão de tipos

In [31]:
columns_with_date_format = []

# Conversão das colunas de datas
for column in df_listings.columns:
    try:
        # Tenta converter os valores da coluna para o formato de data
        pd.to_datetime(df_listings[column], format='%Y-%m-%d', errors='raise')
        # Se a conversão for bem-sucedida, adiciona o nome da coluna à lista
        columns_with_date_format.append(column)
    except ValueError:
        pass
    
df_listings[columns_with_date_format].head()

for col in columns_with_date_format:
    df_listings[col] = pd.to_datetime(df_listings[col], format='%Y-%m-%d', errors='raise')

,last_scraped,host_since,neighbourhood_group_cleansed,bathrooms,calendar_updated,calendar_last_scraped,first_review,last_review,license
0,2023-09-22,2011-09-25,NaN,NaN,NaN,2023-09-22,2011-11-17,2023-09-11,NaN
1,2023-09-22,2011-09-25,NaN,NaN,NaN,2023-09-22,2011-11-02,2016-08-21,NaN
2,2023-09-23,2011-10-03,NaN,NaN,NaN,2023-09-23,2014-03-03,2023-09-05,NaN
3,2023-09-23,2010-01-08,NaN,NaN,NaN,2023-09-23,2010-07-15,2023-09-11,NaN
4,2023-09-22,2010-04-03,NaN,NaN,NaN,2023-09-22,2010-06-07,2023-09-07,NaN


Há colunas que ao invés de possuirem valores booleanos, possuem as strings 'f' e 't'.

In [32]:
columns_f_t = []

# Encontra quais colunas têm 'f' e 't'
for coluna in df_listings.columns:
    if ('f' in df_listings[coluna].astype(str).str.lower().values) and ('t' in df_listings[coluna].astype(str).str.lower().values):
        columns_f_t.append(coluna)

display(df_listings[columns_f_t].head(2))

# Converte para boleano
df_listings[columns_f_t] = df_listings[columns_f_t].applymap({'f': False, 't': True}.get).astype(bool)

display(df_listings[columns_f_t].head(2))

,host_is_superhost,host_has_profile_pic,host_identity_verified,has_availability,instant_bookable
0,f,t,t,t,f
1,f,t,t,t,f


,host_is_superhost,host_has_profile_pic,host_identity_verified,has_availability,instant_bookable
0,False,True,True,True,False
1,False,True,True,True,False


In [42]:
# Seleciona as colunas com textos
string_columns = df_listings.select_dtypes(include=['object']).columns.tolist()
df_listings[string_columns].head(2)

,listing_url,source,name,description,neighborhood_overview,picture_url,host_url,host_name,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_thumbnail_url,host_picture_url,host_neighbourhood,host_verifications,neighbourhood,neighbourhood_cleansed,property_type,room_type,bathrooms_text,amenities,price
0,https://www.airbnb.com/rooms/231497,city scrape,Rental unit in Rio de Janeiro · ★4.73 · 1 bedr...,"This is a big studio at the end of Copacabana,...",NaN,https://a0.muscache.com/pictures/3582382/ee8ac...,https://www.airbnb.com/users/show/1207700,Maria Luiza,"Rio de Janeiro, Brazil","Meu nome é Maria Luiza, adoro ajudar meus hósp...",within a few hours,100%,82%,https://a0.muscache.com/im/users/1207700/profi...,https://a0.muscache.com/im/users/1207700/profi...,Copacabana,"['email', 'phone']",NaN,Copacabana,Entire rental unit,Entire home/apt,1 bath,"[""Hangers"", ""Body soap"", ""Elevator"", ""Bed line...",$180.00
1,https://www.airbnb.com/rooms/231516,city scrape,Rental unit in Rio de Janeiro · ★4.71 · 1 bedr...,"Special location of the building, on Copacaban...",NaN,https://a0.muscache.com/pictures/3671683/d74b4...,https://www.airbnb.com/users/show/1207700,Maria Luiza,"Rio de Janeiro, Brazil","Meu nome é Maria Luiza, adoro ajudar meus hósp...",within a few hours,100%,82%,https://a0.muscache.com/im/users/1207700/profi...,https://a0.muscache.com/im/users/1207700/profi...,Copacabana,"['email', 'phone']",NaN,Copacabana,Entire rental unit,Entire home/apt,NaN,"[""TV"", ""Kitchen"", ""Wifi"", ""Elevator"", ""Air con...",$350.00


In [57]:
# Converte o formato das colunas com 'rate'
rate_columns = ['host_response_rate', 'host_acceptance_rate']

# O que a função lambda está fazendo - apagar depois # bug
def converte_percentage(x):
    if x != np.nan:
        x = str(x).replace('%', '')
        x = float(x) / 100
        return x
    else: 
        return x

df_listings[rate_columns] = df_listings[rate_columns].applymap(lambda x: float(str(x).replace('%', ''))/100 if x != np.nan else x)

In [61]:
# Converte a coluna 'price' para float
df_listings['price'] = df_listings['price'].apply(lambda x: float(str(x).replace('$', '').replace(',', '')) if x != np.nan else x)

In [ ]:
# Cria uma coluna numéricca para bathrooms e descarta a com formato textual
df_listings['bathrooms'] = df_listings['bathrooms_text'].str.extract('(\d+)').astype(float)
df_listings['bathrooms'][:5]

df_listings = df_listings.drop(columns='bathrooms_text')

0    1.0
1    NaN
2    1.0
3    1.0
4    1.0
Name: bathrooms, dtype: float64

Tratamento dos textos

In [120]:
# Seleciona as colunas com textos
string_columns = df_listings.select_dtypes(include=['object']).columns.tolist()
df_listings[string_columns].head(5)

,listing_url,source,name,description,neighborhood_overview,picture_url,host_url,host_name,host_location,host_about,host_response_time,host_thumbnail_url,host_picture_url,host_neighbourhood,host_verifications,neighbourhood,neighbourhood_cleansed,property_type,room_type,bathrooms_text,amenities
0,https://www.airbnb.com/rooms/231497,city scrape,Rental unit in Rio de Janeiro · ★4.73 · 1 bedr...,"This is a big studio at the end of Copacabana,...",NaN,https://a0.muscache.com/pictures/3582382/ee8ac...,https://www.airbnb.com/users/show/1207700,Maria Luiza,"Rio de Janeiro, Brazil","Meu nome é Maria Luiza, adoro ajudar meus hósp...",within a few hours,https://a0.muscache.com/im/users/1207700/profi...,https://a0.muscache.com/im/users/1207700/profi...,Copacabana,"['email', 'phone']",NaN,Copacabana,Entire rental unit,Entire home/apt,1 bath,"[""Hangers"", ""Body soap"", ""Elevator"", ""Bed line..."
1,https://www.airbnb.com/rooms/231516,city scrape,Rental unit in Rio de Janeiro · ★4.71 · 1 bedr...,"Special location of the building, on Copacaban...",NaN,https://a0.muscache.com/pictures/3671683/d74b4...,https://www.airbnb.com/users/show/1207700,Maria Luiza,"Rio de Janeiro, Brazil","Meu nome é Maria Luiza, adoro ajudar meus hósp...",within a few hours,https://a0.muscache.com/im/users/1207700/profi...,https://a0.muscache.com/im/users/1207700/profi...,Copacabana,"['email', 'phone']",NaN,Copacabana,Entire rental unit,Entire home/apt,NaN,"[""TV"", ""Kitchen"", ""Wifi"", ""Elevator"", ""Air con..."
2,https://www.airbnb.com/rooms/236991,city scrape,Rental unit in Rio de Janeiro · ★4.89 · 1 bedr...,"Aconchegante, amplo, básico, arejado, iluminad...","Copacabana, apelidada a princesinha do mar, fa...",https://a0.muscache.com/pictures/5725a59b-147d...,https://www.airbnb.com/users/show/1241662,Nilda,"Rio de Janeiro, Brazil",Hellow ! Im Nilda! I love Rio de Janeiro. ...,within an hour,https://a0.muscache.com/im/pictures/user/fea78...,https://a0.muscache.com/im/pictures/user/fea78...,Copacabana,"['email', 'phone']","Rio de Janeiro, Brazil",Copacabana,Entire rental unit,Entire home/apt,1 bath,"[""Clothing storage: wardrobe"", ""Public or shar..."
3,https://www.airbnb.com/rooms/17878,city scrape,Condo in Rio de Janeiro · ★4.70 · 2 bedrooms ·...,Please note that elevated rates applies for Ne...,This is the one of the bests spots in Rio. Bec...,https://a0.muscache.com/pictures/65320518/3069...,https://www.airbnb.com/users/show/68997,Matthias,"Rio de Janeiro, Brazil",I am a journalist/writer. Lived in NYC for ...,within an hour,https://a0.muscache.com/im/pictures/user/67b13...,https://a0.muscache.com/im/pictures/user/67b13...,Copacabana,"['email', 'phone']","Rio de Janeiro, Brazil",Copacabana,Entire condo,Entire home/apt,1 bath,"[""Patio or balcony"", ""Hangers"", ""Paid parking ..."
4,https://www.airbnb.com/rooms/25026,city scrape,Rental unit in Rio de Janeiro · ★4.71 · 1 bedr...,"**Fully renovated in Dec 2022, new kitchen, n...",Copacabana is a lively neighborhood and the ap...,https://a0.muscache.com/pictures/a745aa21-b8dd...,https://www.airbnb.com/users/show/102840,Viviane,"Rio de Janeiro, Brazil","Hi guys,\n\nViviane is a commercial photograph...",within a few hours,https://a0.muscache.com/im/pictures/user/315dd...,https://a0.muscache.com/im/pictures/user/315dd...,Copacabana,"['email', 'phone']","Rio de Janeiro, Brazil",Copacabana,Entire rental unit,Entire home/apt,1 bath,"[""Clothing storage: wardrobe"", ""Dedicated work..."


In [77]:
string_columns = [col for col in string_columns if ('url' not in col)]

In [ ]:
def clean_text(texto):
    if texto == np.nan:
        return texto
    
    if isinstance(texto, str):
        texto = texto.lower()
        try:
            # Processar o texto com o spaCy
            doc = nlp(texto)
            
            # Extrair tokens e verificar se estão corretos
            tokens_corrigidos = [spell.correction(token.text) for token in doc]
            
            # Juntar os tokens corrigidos de volta em uma string
            texto_corrigido = ' '.join(tokens_corrigidos)
            
            # Remove caracteres especiais
            # bug : essa linha também remove as pontuações
            texto_limpo = re.sub(r'[^a-zA-Z0-9\s]', '', texto_corrigido)
            
            return texto_limpo
        except TypeError:
            return texto

In [125]:
import spacy
import re
from spellchecker import SpellChecker
import numpy as np

# Carrega o modelo de língua portuguesa
nlp_pt = spacy.load('pt_core_news_sm')

# Carrega o modelo de língua inglesa
nlp_en = spacy.load('en_core_web_sm')

# Cria a instância do corretor ortográfico para ambos os idiomas
spell_pt = SpellChecker(language='pt')
spell_en = SpellChecker(language='en')

def clean_text(texto):
    if pd.isnull(texto):
        return texto
    
    if isinstance(texto, str):
        texto = texto.lower()
        
        # Determina o idioma do texto
        doc_pt = nlp_pt(texto)
        doc_en = nlp_en(texto)
        pt_tokens = len([token for token in doc_pt if not token.is_punct])
        en_tokens = len([token for token in doc_en if not token.is_punct])
        
        if pt_tokens > en_tokens:
            nlp = nlp_pt
            spell = spell_pt
        else:
            nlp = nlp_en
            spell = spell_en

        try:
            # Processar o texto com o spaCy
            doc = nlp(texto)
            
            # Extrair tokens e verificar se estão corretos
            tokens_corrigidos = [spell.correction(token.text) for token in doc]
            
            # Juntar os tokens corrigidos de volta em uma string
            texto_corrigido = ' '.join(tokens_corrigidos)
            
            # Remove caracteres especiais
            texto_limpo = re.sub(r'[^a-zA-Z0-9\s]', '', texto_corrigido)
            
            return texto_limpo
        except TypeError:
            return texto



In [126]:
temp = df_listings[string_columns].head().applymap(lambda x: clean_text(x))

In [127]:
temp

,listing_url,source,name,description,neighborhood_overview,picture_url,host_url,host_name,host_location,host_about,host_response_time,host_thumbnail_url,host_picture_url,host_neighbourhood,host_verifications,neighbourhood,neighbourhood_cleansed,property_type,room_type,bathrooms_text,amenities
0,https://www.airbnb.com/rooms/231497,city scrape,rental unit in rio de janeiro · ★4.73 · 1 bedr...,"this is a big studio at the end of copacabana,...",NaN,httpsa0muscachecompictures3582382ee8acc55origi...,httpswwwairbnbcomusersshow1207700,maria liza,"rio de janeiro, brazil","meu nome é maria luiza, adoro ajudar meus hósp...",within a few hours,httpsa0muscachecomimusers1207700profilepic1316...,httpsa0muscachecomimusers1207700profilepic1316...,copacabana,email phone,NaN,copacabana,entire rental unit,entire home apt,1 bath,"[""hangers"", ""body soap"", ""elevator"", ""bed line..."
1,https://www.airbnb.com/rooms/231516,city scrape,rental unit in rio de janeiro · ★4.71 · 1 bedr...,"special location of the building, on copacaban...",NaN,httpsa0muscachecompictures3671683d74b44a4origi...,httpswwwairbnbcomusersshow1207700,maria liza,"rio de janeiro, brazil","meu nome é maria luiza, adoro ajudar meus hósp...",within a few hours,httpsa0muscachecomimusers1207700profilepic1316...,httpsa0muscachecomimusers1207700profilepic1316...,copacabana,email phone,NaN,copacabana,entire rental unit,entire home apt,NaN,to kitchen wifi elevator air con...
2,https://www.airbnb.com/rooms/236991,city scrape,rental unit in rio de janeiro · ★4.89 · 1 bedr...,"aconchegante, amplo, básico, arejado, iluminad...","copacabana, apelidada a princesinha do mar, fa...",httpsa0muscachecompictures5725a59b147d4bf299f2...,httpswwwairbnbcomusersshow1241662,hilda,"rio de janeiro, brazil",hellow ! im nilda! i love rio de janeiro. ...,within an hour,httpsa0muscachecomimpicturesuserfea78163549540...,httpsa0muscachecomimpicturesuserfea78163549540...,copacabana,email phone,"rio de janeiro, brazil",copacabana,entire rental unit,entire home apt,1 bath,"[""clothing storage: wardrobe"", ""public or shar..."
3,https://www.airbnb.com/rooms/17878,city scrape,condo in rio de janeiro · ★4.70 · 2 bedrooms ·...,please note that elevated rates applies for ne...,this is the one of the bests spots in rio bec...,httpsa0muscachecompictures6532051830698f38orig...,httpswwwairbnbcomusersshow68997,matthews,"rio de janeiro, brazil",i i am a i journalist writer lived i in ny f...,within an hour,httpsa0muscachecomimpicturesuser67b13cea8c1149...,httpsa0muscachecomimpicturesuser67b13cea8c1149...,copacabana,email phone,"rio de janeiro, brazil",copacabana,entire condo,entire home apt,1 bath,patio or balcony hangers paid parking ...
4,https://www.airbnb.com/rooms/25026,city scrape,rental unit in rio de janeiro · ★4.71 · 1 bedr...,"**fully renovated in dec 2022, new kitchen, n...",copacabana is a lively neighborhood and the ap...,httpsa0muscachecompicturesa745aa21b8dd4959a040...,httpswwwairbnbcomusersshow102840,vivian,"rio de janeiro, brazil",hi guys i vivian is a commercial photographer...,within a few hours,httpsa0muscachecomimpicturesuser315ddc81bea34b...,httpsa0muscachecomimpicturesuser315ddc81bea34b...,copacabana,email phone,"rio de janeiro, brazil",copacabana,entire rental unit,entire home apt,1 bath,clothing storage wardrobe dedicated work...


In [12]:
string_columns = df_listings.select_dtypes(include=['object']).columns.tolist()
df_listings[string_columns]

,listing_url,last_scraped,source,name,description,neighborhood_overview,picture_url,host_url,host_name,host_since,...,property_type,room_type,bathrooms_text,amenities,price,has_availability,calendar_last_scraped,first_review,last_review,instant_bookable
0,https://www.airbnb.com/rooms/231497,2023-09-22,city scrape,Rental unit in Rio de Janeiro · ★4.73 · 1 bedr...,"This is a big studio at the end of Copacabana,...",NaN,https://a0.muscache.com/pictures/3582382/ee8ac...,https://www.airbnb.com/users/show/1207700,Maria Luiza,2011-09-25,...,Entire rental unit,Entire home/apt,1 bath,"[""Hangers"", ""Body soap"", ""Elevator"", ""Bed line...",$180.00,t,2023-09-22,2011-11-17,2023-09-11,f
1,https://www.airbnb.com/rooms/231516,2023-09-22,city scrape,Rental unit in Rio de Janeiro · ★4.71 · 1 bedr...,"Special location of the building, on Copacaban...",NaN,https://a0.muscache.com/pictures/3671683/d74b4...,https://www.airbnb.com/users/show/1207700,Maria Luiza,2011-09-25,...,Entire rental unit,Entire home/apt,NaN,"[""TV"", ""Kitchen"", ""Wifi"", ""Elevator"", ""Air con...",$350.00,t,2023-09-22,2011-11-02,2016-08-21,f
2,https://www.airbnb.com/rooms/236991,2023-09-23,city scrape,Rental unit in Rio de Janeiro · ★4.89 · 1 bedr...,"Aconchegante, amplo, básico, arejado, iluminad...","Copacabana, apelidada a princesinha do mar, fa...",https://a0.muscache.com/pictures/5725a59b-147d...,https://www.airbnb.com/users/show/1241662,Nilda,2011-10-03,...,Entire rental unit,Entire home/apt,1 bath,"[""Clothing storage: wardrobe"", ""Public or shar...",$190.00,t,2023-09-23,2014-03-03,2023-09-05,f
3,https://www.airbnb.com/rooms/17878,2023-09-23,city scrape,Condo in Rio de Janeiro · ★4.70 · 2 bedrooms ·...,Please note that elevated rates applies for Ne...,This is the one of the bests spots in Rio. Bec...,https://a0.muscache.com/pictures/65320518/3069...,https://www.airbnb.com/users/show/68997,Matthias,2010-01-08,...,Entire condo,Entire home/apt,1 bath,"[""Patio or balcony"", ""Hangers"", ""Paid parking ...",$279.00,t,2023-09-23,2010-07-15,2023-09-11,f
4,https://www.airbnb.com/rooms/25026,2023-09-22,city scrape,Rental unit in Rio de Janeiro · ★4.71 · 1 bedr...,"**Fully renovated in Dec 2022, new kitchen, n...",Copacabana is a lively neighborhood and the ap...,https://a0.muscache.com/pictures/a745aa21-b8dd...,https://www.airbnb.com/users/show/102840,Viviane,2010-04-03,...,Entire rental unit,Entire home/apt,1 bath,"[""Clothing storage: wardrobe"", ""Dedicated work...",$330.00,t,2023-09-22,2010-06-07,2023-09-07,f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31959,https://www.airbnb.com/rooms/985064291460771751,2023-09-22,city scrape,Rental unit in Rio de Janeiro · ★New · 1 bedro...,Relaxe com toda a família nesta acomodação tra...,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,https://www.airbnb.com/users/show/401842074,Nayana,2021-05-16,...,Entire rental unit,Entire home/apt,1 bath,"[""Beach access"", ""Cooking basics"", ""Pool"", ""Po...",$400.00,t,2023-09-22,NaN,NaN,f
31960,https://www.airbnb.com/rooms/985340991466900379,2023-09-22,city scrape,Rental unit in Rio de Janeiro · ★New · 2 bedro...,"Hello!<br /><br />Season apartment, exclusive,...",Copacabana is a 24/7 neighborhood and very cha...,https://a0.muscache.com/pictures/prohost-api/H...,https://www.airbnb.com/users/show/30165706,Yes Temporada,2015-03-28,...,Entire rental unit,Entire home/apt,2 baths,"[""Dedicated workspace"", ""BBQ grill"", ""Hangers""...",$427.00,t,2023-09-22,NaN,NaN,t
31961,https://www.airbnb.com/rooms/985507696630141934,2023-09-22,city scrape,Rental unit in Rio de Janeiro · ★New · 1 bedro...,O Apart-Hotel Villa Del Sol fica localizado na...,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,https://www.airbnb.com/users/show/416337225,Claudio,2021-08-01,...,Entire rental unit,Entire home/apt,1 bath,"[""Beach view"", ""Kitchen"", ""Beach access"", ""TV""...",$479.00,t,2023-09-22,NaN,NaN,f
31962,https://www.airbnb.com/rooms/985510361579508644

Tratamento de duplicados

In [26]:
print(f"df_listings.shape: {df_listings.shape}")

df_listings.shape: (31964, 75)


In [27]:
# Descarta os duplicados
df_listings = df_listings.drop_duplicates()

print(f"df_listings.shape: {df_listings.shape}")

df_listings.shape: (31964, 75)


Tratamento de valores faltantes

In [68]:
# Seleciona as colunas que têm nulos
columns_with_nulls = df_listings.columns[df_listings.isna().any()].tolist()
print(f"Qtd. de colunas com nulos em df_listings: {len(columns_with_nulls)}\n")

percentage_nulls = ((100 * df_listings[columns_with_nulls].isna().sum().sort_values(ascending=False)) / df_listings.shape[0]).reset_index()
percentage_nulls.columns = ['Coluna', '% de nulos']
percentage_nulls

Qtd. de colunas com nulos em df_listings: 36



,Coluna,% de nulos
0,license,100.000000
1,calendar_updated,100.000000
2,bathrooms,100.000000
3,neighbourhood_group_cleansed,100.000000
4,host_about,50.434864
5,neighbourhood,47.631711
6,neighborhood_overview,47.631711
7,review_scores_location,25.963584
8,review_scores_value,25.957327
9,review_scores_communication,25.957327


In [86]:
df_listings = df_listings.drop(columns=['host_picture_url', 'host_thumbnail_url', 'host_has_profile_pic', 'host_total_listings_count', 
'license', 'calendar_updated', 'bathrooms', 'neighbourhood_group_cleansed', 'review_scores_communication', 
'review_scores_accuracy', 'review_scores_checkin', 'review_scores_cleanliness', 'picture_url'])

df_listings = df_listings.drop(columns=[col for col in df_listings.columns if 'url' in col])

 ##### df_reviews

In [133]:
df_reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,231497,726625,2011-11-17,1356783,Jenny,Es un apartastudio muy bien ubicado. Tiene lo ...
1,17878,64852,2010-07-15,135370,Tia,This apartment is in a perfect location -- two...
2,17878,76744,2010-08-11,10206,Mimi,we had a really great experience staying in Ma...
3,17878,91074,2010-09-06,80253,Jan,Staying in Max appartment is like living in a ...
4,17878,137528,2010-11-12,230449,Orene,In general very good and reasonable price.\r<b...


In [158]:
langs = detect_languages(df_reviews, 'comments')

No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features i

KeyboardInterrupt: 

In [156]:
def detect_languages(df, column_name):
    # Obtém uma lista única de comentários da coluna especificada
    comments = df[column_name].dropna().unique()

    # Inicializa um conjunto para armazenar os idiomas detectados
    detected_languages = set()

    for comment in comments:
        try:
            language = detect(comment)
            detected_languages.add(language)
        except Exception as e:
                print(e)
    return detected_languages


In [136]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 637307 entries, 0 to 637306
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   listing_id     637307 non-null  int64         
 1   id             637307 non-null  int64         
 2   date           637307 non-null  datetime64[ns]
 3   reviewer_id    637307 non-null  int64         
 4   reviewer_name  637307 non-null  object        
 5   comments       637288 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 29.2+ MB


Conversão de tipos

In [135]:
df_reviews['date'] = pd.to_datetime(df_reviews['date'], format='%Y-%m-%d', errors='raise')

Descarte de duplicados

In [129]:
print(f"df_reviews.shape: {df_reviews.shape}")

df_reviews.shape: (637307, 6)


In [130]:
df_reviews = df_reviews.drop_duplicates()

In [131]:
print(f"df_reviews.shape: {df_reviews.shape}")

df_reviews.shape: (637307, 6)


Tratamento de nulos

In [132]:
# Seleciona as colunas que têm nulos
columns_with_nulls = df_reviews.columns[df_reviews.isna().any()].tolist()
print(f"Qtd. de colunas com nulos em df_reviews: {len(columns_with_nulls)}\n")

percentage_nulls = ((100 * df_reviews[columns_with_nulls].isna().sum().sort_values(ascending=False)) / df_reviews.shape[0]).reset_index()
percentage_nulls.columns = ['Coluna', '% de nulos']
percentage_nulls

Qtd. de colunas com nulos em df_reviews: 1



,Coluna,% de nulos
0,comments,0.002981


Tratamendo dos texto

In [ ]:
#string_columns = df_listings.select_dtypes(include=['object']).columns.tolist()
# bug Os comentários não estão apenas em pt e en T_T 
df_reviews['reviewer_name', 'comments']	

In [146]:
df_reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,231497,726625,2011-11-17,1356783,Jenny,Es un apartastudio muy bien ubicado. Tiene lo ...
1,17878,64852,2010-07-15,135370,Tia,This apartment is in a perfect location -- two...
2,17878,76744,2010-08-11,10206,Mimi,we had a really great experience staying in Ma...
3,17878,91074,2010-09-06,80253,Jan,Staying in Max appartment is like living in a ...
4,17878,137528,2010-11-12,230449,Orene,In general very good and reasonable price.\r<b...


##### df_calendar

In [139]:
df_calendar.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,17878,2023-09-23,f,$265.00,$265.00,5.0,28.0
1,17878,2023-09-24,t,$265.00,$265.00,5.0,28.0
2,17878,2023-09-25,f,$290.00,$290.00,5.0,28.0
3,17878,2023-09-26,f,$290.00,$290.00,5.0,28.0
4,17878,2023-09-27,f,$290.00,$290.00,5.0,28.0


In [138]:
df_calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11666976 entries, 0 to 11666975
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   listing_id      int64  
 1   date            object 
 2   available       object 
 3   price           object 
 4   adjusted_price  object 
 5   minimum_nights  float64
 6   maximum_nights  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 623.1+ MB


In [137]:
# Descarta os duplicados
df_calendar = df_calendar.drop_duplicates()

print(f"df_calendar.shape: {df_calendar.shape}")

df_calendar.shape: (11666976, 7)


Conversão de tipos

In [141]:
df_calendar['date'] = pd.to_datetime(df_calendar['date'], format='%Y-%m-%d', errors='raise')

df_calendar['price'] = df_calendar['price'].apply(lambda x: float(str(x).replace('$', '').replace(',', '')) if x != np.nan else x)
df_calendar['adjusted_price'] = df_calendar['adjusted_price'].apply(lambda x: float(str(x).replace('$', '').replace(',', '')) if x != np.nan else x)

df_calendar['available'] = df_calendar['available'].map({'f': False, 't': True}.get).astype(bool)

Tratamento de nulos

In [62]:
# Seleciona as colunas que têm nulos
columns_with_nulls = df_calendar.columns[df_calendar.isna().any()].tolist()
print(f"Qtd. de colunas com nulos em df_listings: {len(columns_with_nulls)}\n")

percentage_nulls = ((100 * df_calendar[columns_with_nulls].isna().sum().sort_values(ascending=False)) / df_calendar.shape[0]).reset_index()
percentage_nulls.columns = ['Coluna', '% de nulos']
percentage_nulls

Qtd. de colunas com nulos em df_listings: 2



,Coluna,% de nulos
0,minimum_nights,0.000129
1,maximum_nights,0.000129


In [143]:
df_calendar = df_calendar.fillna(0)

In [145]:
df_calendar.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,17878,2023-09-23,False,265.0,265.0,5.0,28.0
1,17878,2023-09-24,True,265.0,265.0,5.0,28.0
2,17878,2023-09-25,False,290.0,290.0,5.0,28.0
3,17878,2023-09-26,False,290.0,290.0,5.0,28.0
4,17878,2023-09-27,False,290.0,290.0,5.0,28.0


In [142]:
df_calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11666976 entries, 0 to 11666975
Data columns (total 7 columns):
 #   Column          Dtype         
---  ------          -----         
 0   listing_id      int64         
 1   date            datetime64[ns]
 2   available       bool          
 3   price           float64       
 4   adjusted_price  float64       
 5   minimum_nights  float64       
 6   maximum_nights  float64       
dtypes: bool(1), datetime64[ns](1), float64(4), int64(1)
memory usage: 545.2 MB


### 3. **Data Quality - Camada Silver:**
   - Defina métricas de qualidade de dados, como integridade, precisão e consistência para os dados da camada "bronze".
   - Implemente verificações para garantir que os dados da camada "silver" estejam em conformidade com essas métricas.
   - Estabeleça um sistema de monitoramento contínuo da qualidade dos dados da camada "silver".

In [144]:
context = gx.get_context()

### 4. **Testes de Qualidade - Camada Silver:**
   - Utilize a biblioteca Great Expectations para criar testes de qualidade automatizados que verifiquem as expectativas definidas para os dados da camada "silver".
   - Desenvolva testes que assegurem que os dados da camada "silver" atendam às regras de negócios e aos requisitos de qualidade.